In [ ]:
import pandas           as pd
import numpy            as np
import plotly.express   as px
import seaborn         as sns

(pd.set_option('display.float_format', lambda x: '%.3f' % x))

In [2]:
def get_data(path):
    data = pd.read_csv(path)

    return data

path = 'kc_house_data.csv'

data = get_data(path)